# Variational Quantum Circuit

Variational quantum circuit （变分量子线路）是变分量子算法（VQA）和量子神经网络（QNN）的基础，本节将介绍变分量子线路的基本知识和构造方法。

## 概述

含参量子线路（Parameterized Quantum Circuit, PQC）即由含参数的量子门组成的量子线路，是进行量子机器学习的途径之一。在很多情况下，为了能与经典机器学习中神经网络进行类比，我们也经常会把含参量子线路称为量子神经网络。
量子-经典混合计算架构MindQuantum能够处理此类含参量子线路，并利用量子神经网络的可逆性对该线路进行自动微分，最后通过测量得到的观测值，即可计算出观测值对于各参数的导数。

构建PQC并用PQC模拟器算子对量子线路进行演化的大致流程如下：

- **Step 1** ： 初始化量子线路；
- **Step 2** ： 在量子线路中加入所需的含参量子门或者不含参量子门；
- **Step 3** ： 利用PQC模拟器算子进行态演化或者梯度求解。

## 含参量子门

在之前的章节中我们已经学习过如何定义量子线路和量子门，除了 `X`、`Y`、`H` 这种不含参数的量子门，还包括 `RX`、`RY`、`RZ` 这种含参量子门。

In [2]:
from mindquantum.core.circuit import Circuit
from mindquantum.core.gates import X, Y, H, RX, RY, RZ 

circ = Circuit()
circ += X.on(0)
circ += Y.on(1)
circ += H.on(0)
circ += H.on(1)

circ += RX("theta").on(0)
circ += RZ("phi").on(1)

circ.summary()
circ.svg()

===========Circuit Summary===========
|Total number of gates  : 6.        |
|Parameter gates        : 2.        |
|with 2 parameters are  :           |
|theta, phi                        .|
|Number qubit of circuit: 2         |


上述代码定义的量子线路 `circ` 包含 2 个参数门，参数是 `theta` 和 `phi`。通过传入具体的参数值，我们可以得到确定的量子线路：

In [7]:
c0 = circ.apply_value({"theta": 0, "phi": 1})
c0.svg()

## 哈密顿量

在量子计算中，哈密顿量 $H$ 是一个 Hermitian 的算子，即 $H^\dagger = H$ 。求一个量子态 $|\psi\rangle$ 的哈密顿量 $H$ 就是计算内积
$$
\langle \psi | H | \psi \rangle
$$

在 MindQuantum 中，我们可以使用 [mindquantum.core.operators](https://www.mindspore.cn/mindquantum/docs/zh-CN/master/mindquantum.core.operators.html#mindquantum.core.operators.Hamiltonian) 中的 `Hamiltonian` 来定义一个哈密顿量。量子线路常用的哈密顿量使用泡利算符来定义，MindQuantum 使用 `QubitOperator` 来定义泡利算符。

In [9]:
from mindquantum.core.operators import Hamiltonian, QubitOperator

op = QubitOperator("X0 Y1", 0.5)
op += 0.3 * QubitOperator("Z0 X1")
print(f"operator = {op}")

ham = Hamiltonian(op)
print(f"Hamiltonian = {ham}")

operator = 1/2 [X0 Y1] +
3/10 [Z0 X1] 
Hamiltonian = 1/2 [X0 Y1] +
3/10 [Z0 X1] 


在上述代码中，`QubitOperator("X0 Y1", 0.5)` 定义了一个泡利算符 $0.5 X_0Y_1$ ，$X_0$ 表示对 $q_0$ 作用 $X$，$Y_1$ 表示对 $q_1$ 作用 $Y$，`0.5` 是系数。泡利算符可以使用 `+` 来组合，最后使用 `Hamiltonian` 来封装成哈密顿量。

## 模拟器

在 MindQuantum 模拟器相关的 API 在 [mindquantum.simulator](https://www.mindspore.cn/mindquantum/docs/zh-CN/master/mindquantum.simulator.html) 中。

`Simulator` 接受三个参数：

- `backend`：所用到的模拟器后端，最新版本的 `mindquantum` 支持 `projectq`、`mqvector` 和 `mqvector_gpu` 作为后端进行模拟。
- `n_qbuits`：模拟器所用到的比特数，也就是这里的2。
- `seed`：模拟器在运行随机性相关算法时的随机种子，默认为一个随机数，可以不用提供。

MindQuantum 的模拟器是 little endian 的。这里 little endian 的意思是，整个模拟器中，我们都是将比特序号小的比特放在量子态矢量的右边 $|q_{n-1}q_{n-2}\cdots q_1q_0\rangle$。

模拟器的量子态默认（初始化）处于零态 $|0\rangle$。注意，量子模拟器始终会维护一个内部的量子态，当我们作用（`apply`）量子门或者量子线路到模拟器上时，这个量子态会随即发生改变，而当我们只是想获取（`get`）关于这个量子态的一些信息时，这个量子态则不会改变。这里就涉及到对量子模拟器的两类操作：

- 会改变量子态的操作，通常以 `apply` 开头，主要有如下几个
    - `apply_gate`: 作用一个量之门到模拟器上
    - `apply_circuit`: 作用一个量子线路到模拟器上
    - `apply_hamiltonian`: 将一个哈密顿量作用到模拟器上，注意，此后模拟器的量子态将不再是一个真的量子态
    - `set_qs`: 直接设置模拟器的当前量子态
    - `reset`: 重置模拟器的状态为 $|0\rangle$ 态
- 不会改变量子态的操作，通常以 `get` 开头，主要有如下几个
    - `get_qs`: 获取模拟器的当前量子态
    - `get_expectation`: 计算模拟器当前量子态关于某个哈密顿量的期望值
    - `get_expectation_with_grad`: 跟上一个接口类似，只不过这个方法还会计算期望值关于参数化量子线路的梯度
    - `sampling`: 在当前量子态下，对给定的量子线路进行采样

下面的代码计算关于哈密顿量 `ham` 的期望：

In [10]:
from mindquantum.simulator import Simulator

sim = Simulator("mqvector", 2)

print(f"initial state: {sim.get_qs()}")
print(f"expectation: {sim.get_expectation(ham)}")

sim.apply_circuit(circuit=circ, pr={"theta": 0, "phi": 1})
print(f"updated state: {sim.get_qs()}")
print(f"expectation: {sim.get_expectation(ham)}")

initial state: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
expectation: 0j
updated state: [ 0.23971277+0.43879128j -0.23971277-0.43879128j  0.23971277-0.43879128j
 -0.23971277+0.43879128j]
expectation: (0.4207354924039485+0j)


## 训练量子神经网络

`Simulator` 的 `get_expectation_with_grad(hams, circ_right, circ_left=None, simulator_left=None, parallel_worker=None)` 可以返回
$$
E = \langle \varphi | U_l^\dagger H U_r | \psi \rangle
$$
关于线路参数的梯度。

通过使用 `mindspore` 提供的 API 进行训练：

In [11]:
from mindquantum.framework import MQAnsatzOnlyLayer
import mindspore as ms
import mindspore.nn as nn 
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

sim = Simulator('mqvector', circ.n_qubits)                     # 创建模拟器，backend使用‘projectq’，能模拟2个比特（'circ'线路中包含的比特数）
grad_ops = sim.get_expectation_with_grad(ham, circ)            # 获取计算变分量子线路的期望值和梯度的算子
net = MQAnsatzOnlyLayer(grad_ops)                              # 生成待训练的神经网络
opti = nn.Adam(net.trainable_params(), learning_rate=0.05)     # 设置针对网络中所有可训练参数、学习率为0.05的Adam优化器
train_net = nn.TrainOneStepCell(net, opti)                     # 对神经网络进行一步训练

for i in range(100):
    if i % 10 == 0:
        print(f"step {i}: {train_net()}")

step 0: [-0.00266318]
step 10: [-0.02764907]
step 20: [-0.05256478]
step 30: [-0.07734516]
step 40: [-0.10192395]
step 50: [-0.1262336]
step 60: [-0.1502059]
step 70: [-0.17377245]
step 80: [-0.19686492]
step 90: [-0.21941562]


从上述代码运行结果可以看出，的确 `train_net()` 是在不断降低，也就是哈密顿量的期望不断降低。

In [1]:
from show_info import InfoTable

InfoTable('mindquantum', 'mindspore')

Software,Version
mindquantum,0.7.0
mindspore,1.8.1
System,Info
Python,3.9.7
OS,Linux x86_64
Memory,16.72 GB
CPU Max Thread,8
Date,Tue Nov 29 14:23:41 2022
